In [ ]:
!pip install pandas
!pip install seaborn
!pip install librosa
!pip install tensorflow




import tensorflow as tf
if not tf.config.list_physical_devices('GPU'):
    print("GPU not fount install toolkit")
else:
    print("GPU available.")


import os
import random
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (LSTM, Dense, Dropout, Masking, Input, 
                                     Bidirectional, Conv1D, MaxPooling1D, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
N_MFCC = 13
MAX_N_FFT = 2048
BATCH_SIZE = 10

In [ ]:
def load_and_preprocess_data(audio_path, label, n_mfcc=N_MFCC, max_n_fft=MAX_N_FFT):
    try:
        audio_signal, sample_rate = librosa.load(audio_path, sr=None)
        n_fft = min(max_n_fft, len(audio_signal))
        mfccs = librosa.feature.mfcc(y=audio_signal, sr=sample_rate, n_mfcc=n_mfcc, n_fft=n_fft)
        return mfccs.T, label
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None, None


In [ ]:
from tensorflow.keras.utils import Sequence

class AudioDataGenerator(Sequence):
    def __init__(self, real_paths, fake_paths, batch_size, n_mfcc=N_MFCC, max_n_fft=MAX_N_FFT):
        self.real_paths = real_paths
        self.fake_paths = fake_paths
        self.batch_size = batch_size
        self.n_mfcc = n_mfcc
        self.max_n_fft = max_n_fft

        
        min_samples = min(len(real_paths), len(fake_paths))
        self.real_paths = random.sample(real_paths, min_samples)
        self.fake_paths = random.sample(fake_paths, min_samples)

       
        self.data_paths = list(zip(self.real_paths, [0] * len(self.real_paths))) + \
                          list(zip(self.fake_paths, [1] * len(self.fake_paths)))
        random.shuffle(self.data_paths)

    def __len__(self):
        return int(np.ceil(len(self.data_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.data_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        X, y = [], []

        for audio_path, label in batch_data:
            features, label = load_and_preprocess_data(audio_path, label, 
                                                       self.n_mfcc, self.max_n_fft)
            if features is not None:
                X.append(features)
                y.append(label)

        X_padded = pad_sequences(X, padding='post', dtype='float32')
        return np.array(X_padded), np.array(y)


In [ ]:
real_audio_dir = r"C:\\Users\\akn\Desktop\\audio_cllg_hackathons\\audio\\audio\\real"
fake_audio_dir = r"C:\\Users\\akn\\Desktop\\audio_cllg_hackathons\\audio\\audio\\fake"



real_audio_paths = [os.path.join(real_audio_dir, file) for file in os.listdir(real_audio_dir) if file.endswith('.flac')]
fake_audio_paths = [os.path.join(fake_audio_dir, file) for file in os.listdir(fake_audio_dir) if file.endswith('.flac')]


X_train_real, X_val_real = train_test_split(real_audio_paths, test_size=0.2, random_state=42)
X_train_fake, X_val_fake = train_test_split(fake_audio_paths, test_size=0.2, random_state=42)
X_val_real, X_test_real = train_test_split(X_val_real, test_size=0.5, random_state=42)
X_val_fake, X_test_fake = train_test_split(X_val_fake, test_size=0.5, random_state=42)


In [ ]:
train_generator = AudioDataGenerator(X_train_real, X_train_fake, batch_size=BATCH_SIZE)
val_generator = AudioDataGenerator(X_val_real, X_val_fake, batch_size=BATCH_SIZE)
test_generator = AudioDataGenerator(X_test_real, X_test_fake, batch_size=BATCH_SIZE)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
       LSTM(64, return_sequences=True, input_shape=(None, N_MFCC)),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=7,
                    verbose=1)

In [ ]:
model.save('real_batch_lstm_model_akn.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
y_pred = model.predict(test_generator).argmax(axis=1)

y_true = np.concatenate([batch[1] for batch in test_generator], axis=0)


In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
plot_confusion_matrix(y_true, y_pred)
print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))


In [1]:
import numpy as np
from tensorflow.keras.models import load_model
import librosa
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_and_preprocess_data(audio_path, label=None, n_mfcc=13, max_n_fft=2048):
    try:
        audio_signal, sample_rate = librosa.load(audio_path, sr=None)
        n_fft = min(max_n_fft, len(audio_signal))
        mfccs = librosa.feature.mfcc(y=audio_signal, sr=sample_rate, n_mfcc=n_mfcc, n_fft=n_fft)
        return mfccs.T, label
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None, None

In [2]:
def preprocess_unseen_audio(audio_path, n_mfcc=13, max_n_fft=2048):
    features, label = load_and_preprocess_data(audio_path, label=None, n_mfcc=n_mfcc, max_n_fft=max_n_fft)
    if features is not None:
        return features
    else:
        print(f"Error processing {audio_path}")
        return None

In [3]:
model = load_model(r"real_batch_lstm_model_akn.h5")


In [4]:
def predict_audio(audio_path):
    processed_audio = preprocess_unseen_audio(audio_path)

    if processed_audio is not None:
        processed_audio = np.expand_dims(processed_audio, axis=0)
        processed_audio = pad_sequences(processed_audio, padding='post', dtype='float32')
        predictions = model.predict(processed_audio)
        predicted_class = np.argmax(predictions, axis=1)[0]

        if predicted_class == 0:
            print("The audio is predicted to be real.")
        else:
            print("The audio is predicted to be fake.")

        print("Prediction probabilities:", predictions)
    else:
        print("Error: Failed to preprocess audio.")

In [3]:
audio_file_path = r"C:\Users\akn\Desktop\Audio\real\PA_E_2541263.flac"
predict_audio(audio_file_path)

NameError: name 'predict_audio' is not defined

In [ ]:
# model.save('batch_lstm_model_2.h5')